In [84]:
import pandas as pd
import numpy as np
# from pycontractions import Contractions
# import gensim

# Preprocessing
from datetime import datetime
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Modelling
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report

# Deep Learning Modelling
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU, Dropout, Bidirectional, SpatialDropout1D

In [58]:
# Function to parse date
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

# Read CSV while parsing the dates
opinion_df = pd.read_csv('./Data/all_opinions.csv', parse_dates=['date_filed'], date_parser=dateparse)

# Get head of opinion DF
opinion_df.head()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,There is no right more basic in our democracy ...
1,Justice Thomas,concurring,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,I adhere to the view that this Court’s decisio...
2,Justice Breyer,dissenting,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,"Nearly 40 years ago in Buckley v. Valeo, 424 U..."
3,Justice Taney,majority,False,Ex Parte Crenshaw,1841-02-18,40 U.S. 119,https://www.courtlistener.com/opinion/86166/ex...,https://www.courtlistener.com/api/rest/v3/clus...,1841,1841-005,2.0,9.0,0.0,This case was brought here by an appeal from t...
4,Justice Pitney,majority,False,Richards v. Washington Terminal Co.,1914-05-04,233 U.S. 546,https://www.courtlistener.com/opinion/98178/ri...,https://www.courtlistener.com/api/rest/v3/clus...,1914,1913-149,1.0,8.0,1.0,"Plaintiff in error, who was plaintiff below, c..."


# Data Cleaning

In [59]:
# Copy the main df
opinion_copy = opinion_df.copy()

# For Reference
print(opinion_copy.columns)

# print(opinion_copy.dtypes)

Index(['author_name', 'category', 'per_curiam', 'case_name', 'date_filed',
       'federal_cite_one', 'absolute_url', 'cluster', 'year_filed', 'scdb_id',
       'scdb_decision_direction', 'scdb_votes_majority', 'scdb_votes_minority',
       'text'],
      dtype='object')


In [60]:
# Get opinions in the past 50 years
above_1970 = opinion_copy[opinion_copy['date_filed'] > "1970-01-01"]

# Check the value
above_1970

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,There is no right more basic in our democracy ...
1,Justice Thomas,concurring,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,I adhere to the view that this Court’s decisio...
2,Justice Breyer,dissenting,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,"Nearly 40 years ago in Buckley v. Valeo, 424 U..."
16,Justice Kagan,majority,False,Kaley v. United States,2014-02-25,NaN,https://www.courtlistener.com/opinion/2654533/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,NaN,NaN,NaN,NaN,"A federal statute, 21 U.S. C. §853(e), authori..."
17,Justice Roberts,dissenting,False,Kaley v. United States,2014-02-25,NaN,https://www.courtlistener.com/opinion/2654533/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,NaN,NaN,NaN,NaN,An individual facing serious criminal charges ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35776,Justice O'Connor,majority,False,Pilot Life Ins. Co. v. Dedeaux,1987-04-06,NaN,https://www.courtlistener.com/opinion/111858/p...,https://www.courtlistener.com/api/rest/v3/clus...,1987,1986-067,2.0,9.0,0.0,This case presents the question whether the Em...
35777,Justice Powell,majority,False,"Gertz v. Robert Welch, Inc.",1974-06-25,NaN,https://www.courtlistener.com/opinion/109091/g...,https://www.courtlistener.com/api/rest/v3/clus...,1974,1973-162,1.0,7.0,2.0,This Court has struggled for nearly a decade t...
35778,Justice Blackmun,concurring,False,"Gertz v. Robert Welch, Inc.",1974-06-25,NaN,https://www.courtlistener.com/opinion/109091/g...,https://www.courtlistener.com/api/rest/v3/clus...,1974,1973-162,1.0,7.0,2.0,I joined MR. JUSTICE BRENNAN'S opinion for the...
35779,Justice Burger,dissenting,False,"Gertz v. Robert Welch, Inc.",1974-06-25,NaN,https://www.courtlistener.com/opinion/109091/g...,https://www.courtlistener.com/api/rest/v3/clus...,1974,1973-162,1.0,7.0,2.0,The doctrines of the law of defamation have ha...


In [61]:
# Remove Justice Douglas given how his opinions is highly unusual
# Refer to https://www.thenation.com/article/archive/tragedy-william-o-douglas/
above_1970_no_douglas = above_1970[above_1970['author_name'] != 'Justice Douglas']

# Remove those texts with less than 3000 characters as these are recounting past opinions
# Refer to https://www.kaggle.com/gqfiddler/scotus-opinions description of the dataset
char_above3000_1970 = above_1970_no_douglas[above_1970_no_douglas['text'].str.len() > 3000]

# Checking values
char_above3000_1970

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,There is no right more basic in our democracy ...
1,Justice Thomas,concurring,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,I adhere to the view that this Court’s decisio...
2,Justice Breyer,dissenting,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,"Nearly 40 years ago in Buckley v. Valeo, 424 U..."
16,Justice Kagan,majority,False,Kaley v. United States,2014-02-25,NaN,https://www.courtlistener.com/opinion/2654533/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,NaN,NaN,NaN,NaN,"A federal statute, 21 U.S. C. §853(e), authori..."
17,Justice Roberts,dissenting,False,Kaley v. United States,2014-02-25,NaN,https://www.courtlistener.com/opinion/2654533/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,NaN,NaN,NaN,NaN,An individual facing serious criminal charges ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35767,Justice Kagan,majority,False,Match-E-Be-Nash-She-Wish Band of Pottawatomi I...,2012-06-18,NaN,https://www.courtlistener.com/opinion/802400/m...,https://www.courtlistener.com/api/rest/v3/clus...,2012,2011-065,1.0,8.0,1.0,A provision of the Indian Reorganization Act (...
35768,Justice Sotomayor,dissenting,False,Match-E-Be-Nash-She-Wish Band of Pottawatomi I...,2012-06-18,NaN,https://www.courtlistener.com/opinion/802400/m...,https://www.courtlistener.com/api/rest/v3/clus...,2012,2011-065,1.0,8.0,1.0,"In enacting the Quiet Title Act (QTA), Congres..."
35776,Justice O'Connor,majority,False,Pilot Life Ins. Co. v. Dedeaux,1987-04-06,NaN,https://www.courtlistener.com/opinion/111858/p...,https://www.courtlistener.com/api/rest/v3/clus...,1987,1986-067,2.0,9.0,0.0,This case presents the question whether the Em...
35777,Justice Powell,majority,False,"Gertz v. Robert Welch, Inc.",1974-06-25,NaN,https://www.courtlistener.com/opinion/109091/g...,https://www.courtlistener.com/api/rest/v3/clus...,1974,1973-162,1.0,7.0,2.0,This Court has struggled for nearly a decade t...


In [62]:
# Drop values that are not relevant for our analysis
to_analyze = char_above3000_1970.drop(columns=['absolute_url', 'cluster',
                                      'scdb_id', 'federal_cite_one',
                                       'scdb_decision_direction', 'scdb_votes_majority', 'scdb_votes_minority'])
# Check the new_df
to_analyze = to_analyze.reset_index(drop=True)

# Feature Engineering

1. Convert the per_curiam into target variables of 0 and 1
2. Convert the category into categorical variables
3. Text Preprocessing
4. Handle Unbalanced Data Issue

In [63]:
# Step 1: Convert the per_curiam to target variables of 0 and 1

# Function to convert to 0 and 1
def label(x):
    if x == 'False':
        return 0
    return 1

# Change the per curiam to be of string
to_analyze['per_curiam'] = to_analyze['per_curiam'].astype('str')

# Apply the values throughout
to_analyze['per_curiam_label'] = to_analyze['per_curiam'].apply(lambda x: label(x))

# Check the values
to_analyze['per_curiam_label'].value_counts()

0    8555
1     611
Name: per_curiam_label, dtype: int64

As seen above there is an issue with unbalanced data given we would have to address this issue later

In [64]:
# Step 2: Encode the category column variables

# Get the unique values
categories = to_analyze['category'].unique()

# Encoding dictionary
cat_dict = {}

# Store the keys in the dictionary
for i in range(len(categories)):
    cat_dict[categories[i]] = i
    
# Apply the values throughout
to_analyze['cat_labels'] = to_analyze['category'].apply(lambda x: cat_dict[x])

# Check the values
to_analyze['cat_labels'].value_counts()

0    4778
2    2330
1    1103
4     523
3     432
Name: cat_labels, dtype: int64

## Text Preprocessing

Steps to clean

1. Standardize punctuations
2. Remove redundant words first
    - Key things to remove:
        1. Cite as: .. (Indicates how to cite the case) (Regex to get)
        2. Previous citations (Indicates where to find his substantiation of his points) (Remove the ones that start with capital See)
        3. Current case name (lower and remove)
3. Remove unicode characters
4. Remove breakline characters

Steps to preprocess
1. Lower case
2. Remove punctuations
3. Remove stopwords
4. Stem/Lemmatize

In [65]:
# Function to preprocess text

def clean(x):
    
    # Split the case name into array for individual capitalizing
    case_name_array = x['case_name'].split()
    
    # Iterate through the words to capitalize
    for i in range(len(case_name_array)):
        
        # If word not versus, capitalize it to remove later
        if case_name_array[i] != 'v.':
            case_name_array[i] = case_name_array[i].upper()
    
    # Join the case name array together
    case_name = ' '.join(case_name_array)
    
    # 1. Standardizing some punctuations
    tmp = x['text'].replace('’', "'")
#     tmp = tmp.replace('“', '"')
#     tmp = tmp.replace('”', '"')
    tmp = tmp.replace('–', "-")
    tmp = re.sub(r'([.]\s+){2,10}', '', tmp)
    tmp = tmp.replace('[', '')
    tmp = tmp.replace(']', '')
    
    # 2. Remove redundant words
    
    # A. (i) Remove Cite as: since these are words that keep appearing at the bottom of the transcript for citation
    tmp = re.sub(r'Cite as:(.*?)\((\d{4})\)', '', tmp)
    
    # A. (ii) Remove Opinion of Justice given it is a demarcation of the transcript
    tmp = re.sub(r'Opinion\s(.*?)\n', '', tmp)
    
    # B. Remove Case Name
    tmp = re.sub(f'''{case_name}''', '', tmp)
    
    #B. Remove See ... (As these are citations of previous cases to be used)
    # Three Cases
    # i) See case and citation
    see_pattern = re.compile(r"See(.*?)(\)\.|\)\;|\d\.)", re.DOTALL)
    tmp = re.sub(see_pattern, '', tmp)
    
    # (ii) Remove quotes i.e. Herring v. New York, 422 U.S. 853, 862 (1975)
#     tmp = re.sub(r'', '', tmp)

#     print(tmp)
    
    # 3. Remove unicode characters in text
    tmp = re.sub(r'''\\x[a-zA-z0-9]{2}''', '', tmp)
    
    # 4. Remove breakline in text
    
    # A. Embedded in the string (-\\n)
    tmp = re.sub(r'-\n\s{1,}', '', tmp)
    tmp = re.sub(r'-\n', '', tmp) 
    
    # B. Remove long breaks
    tmp = re.sub(r'\n\s+', ' ', tmp)
    
    # C. Remove the remaining breaklines
    tmp = re.sub(r'\n', ' ', tmp)
    
    # 5. Remove Numbers that demarcate sections of opinions
    tmp = re.sub(r'\s\d{1,}\s{2,}', ' ', tmp)
    
    return tmp

In [66]:
to_analyze['cleaned_text'] = to_analyze.apply(lambda x : clean(x), axis=1)

In [67]:
to_analyze.to_excel('for_nicole.xlsx', index=False)

In [68]:
# Instantiate the set of stopwords
stopwords_set = set(stopwords.words('english'))

# Instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess(x):
    
    # 1. Lower case
    tmp = x.lower()
    
    # 2. Remove punctuations
    tmp = tmp.translate(str.maketrans('', '', string.punctuation))
    
    # 3. Tokenize the sentences
    tokens = word_tokenize(tmp)
    
    # 4. Remove stopwords
    no_stopwords = [word for word in tokens if word not in stopwords_set and word.isalpha()]
    
    # 5. Lemmatize
    lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in no_stopwords])
    
    return lemma_text

In [69]:
to_analyze['preprocessed_text'] = to_analyze['cleaned_text'].apply(lambda x: preprocess(x))

to_analyze['preprocess_no_clean_text'] = to_analyze['text'].apply(lambda x: preprocess(x))

to_analyze

,author_name,category,per_curiam,case_name,date_filed,year_filed,text,per_curiam_label,cat_labels,cleaned_text,preprocessed_text,preprocess_no_clean_text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,2014,There is no right more basic in our democracy ...,0,0,There is no right more basic in our democracy ...,right basic democracy right participate electi...,right basic democracy right participate electi...
1,Justice Thomas,concurring,False,McCutcheon v. Federal Election Comm'n,2014-04-02,2014,I adhere to the view that this Court’s decisio...,0,1,I adhere to the view that this Court's decisio...,adhere view court decision buckley v valeo u p...,adhere view court decision buckley v valeo u p...
2,Justice Breyer,dissenting,False,McCutcheon v. Federal Election Comm'n,2014-04-02,2014,"Nearly 40 years ago in Buckley v. Valeo, 424 U...",0,2,"Nearly 40 years ago in Buckley v. Valeo, 424 U...",nearly year ago buckley v valeo u per curiam c...,nearly year ago buckley v valeo u per curiam c...
3,Justice Kagan,majority,False,Kaley v. United States,2014-02-25,2014,"A federal statute, 21 U.S. C. §853(e), authori...",0,0,"A federal statute, 21 U.S. C. 853(e), authoriz...",federal statute u c authorizes court freeze in...,federal statute u c authorizes court freeze in...
4,Justice Roberts,dissenting,False,Kaley v. United States,2014-02-25,2014,An individual facing serious criminal charges ...,0,2,An individual facing serious criminal charges ...,individual facing serious criminal charge brou...,individual facing serious criminal charge brou...
...,...,...,...,...,...,...,...,...,...,...,...,...
9161,Justice Kagan,majority,False,Match-E-Be-Nash-She-Wish Band of Pottawatomi I...,2012-06-18,2012,A provision of the Indian Reorganization Act (...,0,0,A provision of the Indian Reorganization Act (...,provision indian reorganization act ira u c au...,provision indian reorganization act ira u c au...
9162,Justice Sotomayor,dissenting,False,Match-E-Be-Nash-She-Wish Band of Pottawatomi I...,2012-06-18,2012,"In enacting the Quiet Title Act (QTA), Congres...",0,2,"In enacting the Quiet Title Act (QTA), Congres...",enacting quiet title act qta congress waived g...,enacting quiet title act qta congress waived g...
9163,Justice O'Connor,majority,False,Pilot Life Ins. Co. v. Dedeaux,1987-04-06,1987,This case presents the question whether the Em...,0,0,This case presents the question whether the Em...,case present question whether employee retirem...,case present question whether employee retirem...
9164,Justice Powell,majority,False,"Gertz v. Robert Welch, Inc.",1974-06-25,1974,This Court has struggled for nearly a decade t...,0,0,This Court has struggled for nearly a decade t...,court struggled nearly decade define proper ac...,court struggled nearly decade define proper ac...


# Modelling

### Support Vector Machine (TF IDF Vectorizer with Regex Removal)

In [70]:
# Prepare the training and test set
X_train, X_test, y_train, y_test = train_test_split(to_analyze['preprocessed_text'],to_analyze['per_curiam_label'],test_size=0.2, random_state=424)

In [71]:
# Convert words into TF-IDF vectors

# Instantiate TF-IDF Vectorizer
Tfidf_vect = TfidfVectorizer()

# Fit the list into the vectorizer
Tfidf_vect.fit(to_analyze['preprocessed_text'])

# Transform the words into TF-IDF vectors
X_train_Tfidf = Tfidf_vect.transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

In [72]:
# Instantiate and train SVM 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_Tfidf,y_train)

# predict the labels on validation dataset
y_pred = SVM.predict(X_test_Tfidf)

# Use accuracy_score function to get the accuracy
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1714
           1       0.86      0.21      0.34       120

    accuracy                           0.95      1834
   macro avg       0.90      0.60      0.65      1834
weighted avg       0.94      0.95      0.93      1834



### Support Vector Machine (Count Vectorizer with Regex Removal)

In [73]:
# Convert words into TF-IDF vectors

# Instantiate TF-IDF Vectorizer
count_vect = CountVectorizer()

# Fit the list into the vectorizer
count_vect.fit(to_analyze['preprocessed_text'])

# Transform the words into TF-IDF vectors
X_train_cv = count_vect.transform(X_train)
X_test_cv = count_vect.transform(X_test)

In [74]:
# Instantiate and train SVM 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_cv,y_train)

# predict the labels on validation dataset
y_pred = SVM.predict(X_test_cv)

# Use accuracy_score function to get the accuracy
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1714
           1       0.56      0.58      0.57       120

    accuracy                           0.94      1834
   macro avg       0.77      0.78      0.77      1834
weighted avg       0.94      0.94      0.94      1834



### Support Vector Machine (TFIDF Vectorizer With General Preprocessing)

In [75]:
# Prepare the training and test set
X1_train = to_analyze['preprocess_no_clean_text'][X_train.index]
X1_test = to_analyze['preprocess_no_clean_text'][X_test.index] 
y1_train = to_analyze['per_curiam_label'][y_train.index] 
y1_test = to_analyze['per_curiam_label'][y_test.index]

In [76]:
# Convert words into TF-IDF vectors

# Instantiate TF-IDF Vectorizer
Tfidf_vect = TfidfVectorizer()

# Fit the list into the vectorizer
Tfidf_vect.fit(to_analyze['preprocess_no_clean_text'])

# Transform the words into TF-IDF vectors
X1_train_Tfidf = Tfidf_vect.transform(X1_train)
X1_test_Tfidf = Tfidf_vect.transform(X1_test)

In [77]:
# Instantiate and train SVM 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X1_train_Tfidf,y1_train)

# predict the labels on validation dataset
y1_pred = SVM.predict(X1_test_Tfidf)

# Use accuracy_score function to get the accuracy
print(classification_report(y1_test, y1_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1714
           1       0.87      0.22      0.35       120

    accuracy                           0.95      1834
   macro avg       0.91      0.61      0.66      1834
weighted avg       0.94      0.95      0.93      1834



### Support Vector Machine (Count Vectorizer with General Preprocessing)

In [78]:
# Convert words into TF-IDF vectors

# Instantiate TF-IDF Vectorizer
count_vect = CountVectorizer()

# Fit the list into the vectorizer
count_vect.fit(to_analyze['preprocess_no_clean_text'])

# Transform the words into TF-IDF vectors
X1_train_cv = count_vect.transform(X1_train)
X1_test_cv = count_vect.transform(X1_test)

In [79]:
# Instantiate and train SVM 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X1_train_cv,y1_train)

# predict the labels on validation dataset
y1_pred = SVM.predict(X1_test_cv)

# Use accuracy_score function to get the accuracy
print(classification_report(y1_test, y1_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1714
           1       0.57      0.58      0.58       120

    accuracy                           0.94      1834
   macro avg       0.77      0.78      0.77      1834
weighted avg       0.94      0.94      0.94      1834



### GRU with Regex Removal

In [80]:
# For reference to be considered for max sequence length
from statistics import mean
# X_train.apply(lambda x : len(x.split(' '))).quantile(0.95)
mean([len(x.split()) for x in to_analyze['preprocessed_text'].values])

1777.18765001091

In [89]:
# Initial Parameters
vocab_size = 50000
embedding_dim = 128
max_length = 1800
target_length = 2 

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words = vocab_size)

# Fit the tokenizer on the text
tokenizer.fit_on_texts(to_analyze['preprocessed_text'])

# Get the word indexes for the tokens
word_index = tokenizer.word_index

# Convert the train and test texts to sequences and pad them
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_length)

In [91]:
# Modelling

# Instantiate Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(128)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(target_length, activation='softmax'))

# Compile Model
model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(train_padded, y_train, epochs=5, 
                    batch_size=128)

Epoch 1/5
58/58 [==============================] - 3629s 63s/step - loss: 0.2973 - accuracy: 0.9240
Epoch 2/5
58/58 [==============================] - 3860s 67s/step - loss: 0.1470 - accuracy: 0.9501
Epoch 3/5
58/58 [==============================] - 4018s 69s/step - loss: 0.0814 - accuracy: 0.9703
Epoch 4/5
58/58 [==============================] - 4143s 71s/step - loss: 0.0377 - accuracy: 0.9885
Epoch 5/5
58/58 [==============================] - 4240s 73s/step - loss: 0.0143 - accuracy: 0.9952


In [92]:
# Predict test data
y_sub_hat_ = model.predict(test_padded)
y_nn_pred = [np.argmax(x) for x in y_sub_hat_]

# Use classification report function to get confusion matrix
print(classification_report(y_test, y_nn_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1714
           1       0.60      0.50      0.55       120

    accuracy                           0.95      1834
   macro avg       0.78      0.74      0.76      1834
weighted avg       0.94      0.95      0.94      1834



### GRU with General Preprocessing

In [ ]:
mean([len(x.split()) for x in to_analyze['preprocess_no_clean_text'].values])

In [93]:
# Initial Parameters
vocab_size = 50000
embedding_dim = 128
max_length = 1900
target_length = 2 

# Instantiate the Tokenizer
tokenizer = Tokenizer(num_words = vocab_size)

# Fit the tokenizer on the text
tokenizer.fit_on_texts(to_analyze['preprocess_no_clean_text'])

# Get the word indexes for the tokens
word_index = tokenizer.word_index

# Convert the train and test texts to sequences and pad them
train1_sequences = tokenizer.texts_to_sequences(X1_train)
train1_padded = pad_sequences(train1_sequences, maxlen=max_length)

test1_sequences = tokenizer.texts_to_sequences(X1_test)
test1_padded = pad_sequences(test1_sequences, maxlen=max_length)

In [94]:
# Modelling

# Instantiate Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(128)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(target_length, activation='softmax'))

# Compile Model
model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(train1_padded, y1_train, epochs=5, 
                    batch_size=128)

Epoch 1/5
58/58 [==============================] - 4626s 80s/step - loss: 0.2897 - accuracy: 0.9289
Epoch 2/5
58/58 [==============================] - 4726s 81s/step - loss: 0.1427 - accuracy: 0.9501
Epoch 3/5
58/58 [==============================] - 4807s 83s/step - loss: 0.0770 - accuracy: 0.9715
Epoch 4/5
58/58 [==============================] - 4854s 84s/step - loss: 0.0331 - accuracy: 0.9885A: 3
Epoch 5/5
58/58 [==============================] - 5079s 88s/step - loss: 0.0098 - accuracy: 0.9970


In [95]:
# Predict test data
y1_sub_hat_ = model.predict(test1_padded)
y1_nn_pred = [np.argmax(x) for x in y1_sub_hat_]

# Use classification report function to get confusion matrix
print(classification_report(y1_test, y1_nn_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1714
           1       0.77      0.40      0.53       120

    accuracy                           0.95      1834
   macro avg       0.87      0.70      0.75      1834
weighted avg       0.95      0.95      0.95      1834



In [ ]:
to_analyze[['cat_labels', 'category']]